[How to pass variable value to a macro in julia?](https://stackoverflow.com/questions/47977414/how-to-pass-variable-value-to-a-macro-in-julia)

In [22]:
macro m(a,b)
    # @show a,b
    ex = quote
      $a = eval($b)
    end
    # @show ex
    esc(ex)
  end
  
  b = 1
  @m a $b

ErrorException: syntax: "$" expression outside quote around e:\JuliaProjects\Training3.jl\macro3\31.ipynb:4

Macros can have type annotations too, they are functions that **work at compile time, but their inputs can only be Exprs, Symbols or constant values,** ie: Int, macros don't evaluate their inputs before being invoked, like functions do before being called, macros work on syntax.

**Do not use eval inside the body of a macro,** in this case it is in the body of the returned expression (there are cases when this could be needed, but this is not one of them).

In [25]:
macro m(a::Symbol, b)  # in ths method a should allways be a symbol
    # use spaces and parens as needed to delimit and 
    # group args as needed, not commas in this case
    @show a b          # or use this: @show(a, b) notice comma here
    ex = quote         # there is no need to use eval here a macro
        $a = $b        # will compile and evaluate the returning
    end                # expression when invoked
    @show ex
    return esc(ex)     # esc will use surrounding scope instead of gensyms
end
@m y (x + 1)  # parens for grouping, or @m(y, x + 1)

a = :y
b = :(x + 1)
ex = quote
    #= e:\JuliaProjects\Training3.jl\macro3\31.ipynb:6 =#
    y = x + 1
end


UndefVarError: UndefVarError: `x` not defined

In [26]:
x=2
@m y (x + 1)

a = :y
b = :(x + 1)
ex = quote
    #= e:\JuliaProjects\Training3.jl\macro3\31.ipynb:6 =#
    y = x + 1
end


3

In [30]:
macro m(a::Symbol, b)  # in ths method a should allways be a symbol
    # use spaces and parens as needed to delimit and 
    # group args as needed, not commas in this case
    @show a b          # or use this: @show(a, b) notice comma here
    ex = quote         # there is no need to use eval here a macro
        $a = $b        # will compile and evaluate the returning
    end                # expression when invoked
    @show ex
    return ex    # esc will use surrounding scope instead of gensyms
end
@m y (x + 1)  # parens for grouping, or @m(y, x + 1)

a = :y
b = :(x + 1)
ex = quote
    #= e:\JuliaProjects\Training3.jl\macro3\31.ipynb:6 =#
    y = x + 1
end


3

If you don't use esc it will hygienically create gensyms, in this case it uses the surrounding scope variables instead.

In [31]:
@m z rand(x, y)

a = :z
b = :(rand(x, y))
ex = quote
    #= e:\JuliaProjects\Training3.jl\macro3\31.ipynb:6 =#
    z = rand(x, y)
end


2×3 Matrix{Float64}:
 0.0351479  0.170064  0.20941
 0.695855   0.379596  0.287176

In [1]:
gensym(:foo)

Symbol("##foo#225")